# AFL Data Cleaning for Prediction Modeling

### Data found on Kaggle [here](https://www.kaggle.com/datasets/stoney71/aflstats)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import ttest_ind, zscore

%matplotlib inline

#Supresses scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [2]:
games_df = pd.read_csv("games.csv")
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2024 entries, 0 to 2023
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gameId         2024 non-null   object 
 1   year           2024 non-null   int64  
 2   round          2024 non-null   object 
 3   date           2024 non-null   object 
 4   venue          2024 non-null   object 
 5   startTime      2024 non-null   object 
 6   attendance     2024 non-null   int64  
 7   homeTeam       2024 non-null   object 
 8   homeTeamScore  2024 non-null   int64  
 9   awayTeam       2024 non-null   object 
 10  awayTeamScore  2024 non-null   int64  
 11  rainfall       1993 non-null   float64
dtypes: float64(1), int64(4), object(7)
memory usage: 189.9+ KB


In [3]:
players_df = pd.read_csv("players.csv")
players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   playerId     1495 non-null   int64 
 1   displayName  1495 non-null   object
 2   height       1495 non-null   int64 
 3   weight       1495 non-null   int64 
 4   dob          1495 non-null   object
 5   position     1495 non-null   object
 6   origin       1491 non-null   object
dtypes: int64(3), object(4)
memory usage: 81.9+ KB


In [4]:
stats_df = pd.read_csv("stats.csv")
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89470 entries, 0 to 89469
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   gameId                   89470 non-null  object
 1   team                     89470 non-null  object
 2   year                     89470 non-null  int64 
 3   round                    89470 non-null  object
 4   playerId                 89470 non-null  int64 
 5   displayName              89470 non-null  object
 6   gameNumber               89470 non-null  int64 
 7   Disposals                89470 non-null  int64 
 8   Kicks                    89470 non-null  int64 
 9   Marks                    89470 non-null  int64 
 10  Handballs                89470 non-null  int64 
 11  Goals                    89470 non-null  int64 
 12  Behinds                  89470 non-null  int64 
 13  Hit Outs                 89470 non-null  int64 
 14  Tackles                  89470 non-nul

## Merging dataframes to create a dataframe broken down by player and game

In [5]:
df = stats_df.merge(games_df, how='left', on='gameId')\
             .merge(players_df, how='left', on='playerId')
df

,gameId,team,year_x,round_x,playerId,displayName_x,gameNumber,Disposals,Kicks,Marks,Handballs,Goals,Behinds,Hit Outs,Tackles,Rebounds,Inside 50s,Clearances,Clangers,Frees,Frees Against,Brownlow Votes,Contested Possessions,Uncontested Possessions,Contested Marks,Marks Inside 50,One Percenters,Bounces,Goal Assists,% Played,Subs,year_y,round_y,date,venue,startTime,attendance,homeTeam,homeTeamScore,awayTeam,awayTeamScore,rainfall,displayName_y,height,weight,dob,position,origin
0,2021R104,Adelaide,2021,R1,2021661124,"Berry, Sam",1,8,6,1,2,0,1,0,7,0,2,3,2,2,2,0,4,5,0,0,0,0,1,80,-,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.00,"Berry, Sam",181,81,12-Feb-2002,"Midfield, Forward",Gippsland Power
1,2021R104,Adelaide,2021,R1,2012662083,"Brown, Luke",168,5,2,0,3,0,0,0,2,0,0,0,0,1,0,0,2,3,0,0,0,0,0,23,Off,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.00,"Brown, Luke",181,81,22-Sep-1992,Defender,Norwood
2,2021R104,Adelaide,2021,R1,2020665315,"Butts, Jordon",3,10,5,3,5,0,0,0,0,5,0,0,3,1,0,0,5,5,1,0,8,0,0,93,-,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.00,"Butts, Jordon",198,83,31-Dec-1999,Defender,Murray Bushrangers
3,2021R104,Adelaide,2021,R1,2018689604,"Doedee, Tom",31,13,9,4,4,0,0,0,4,5,1,1,0,2,0,0,8,6,0,0,7,0,0,84,-,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.00,"Doedee, Tom",188,88,1-Mar-1997,Defender,Geelong Falcons
4,2021R104,Adelaide,2021,R1,2018703883,"Frampton, Billy",9,14,10,8,4,2,2,7,0,2,0,0,1,0,0,0,5,9,3,4,2,0,0,90,-,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.00,"Frampton, Billy",200,90,20-Nov-1996,Forward,South Fremantle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89465,2012R2309,Western Bulldogs,2012,R23,2011838967,"Skinner, Zephaniah",8,2,1,1,1,0,0,0,1,0,1,0,1,0,0,0,2,1,0,0,1,0,0,28,On,2012,R23,02-Sep-2012,Gabba,4:40 PM,18289,Brisbane Lions,128,Western Bulldogs,61,0.00,"Skinner, Zephaniah",189,79,27-Jun-1989,Forward,-
89466,2012R2309,Western Bulldogs,2012,R23,2012833342,"Smith, Clay",16,22,11,5,11,0,0,0,4,2,3,0,5,2,1,0,6,16,0,0,1,0,0,79,-,2012,R23,02-Sep-2012,Gabba,4:40 PM,18289,Brisbane Lions,128,Western Bulldogs,61,0.00,"Smith, Clay",181,85,11-May-1993,Forward,Gippsland Power
89467,2012R2309,Western Bulldogs,2012,R23,2012846405,"Talia, Michael",4,23,14,9,9,0,0,0,1,1,3,0,3,3,2,0,5,18,0,0,3,4,0,88,-,2012,R23,02-Sep-2012,Gabba,4:40 PM,18289,Brisbane Lions,128,Western Bulldogs,61,0.00,"Talia, Michael",194,94,11-Feb-1993,Defender,Calder Cannons
89468,2012R2309,Western Bulldogs,2012,R23,2011872415,"Wallis, Mitch",25,19,8,2,11,1,0,0,4,1,3,6,2,3,0,0,9,11,0,0,2,0,0,83,-,2012,R23,02-Sep-2012,Gabba,4:40 PM,18289,Brisbane Lions,128,Western Bulldogs,61,0.00,"Wallis, Mitch",186,85,24-Oct-1992,Forward,Calder Cannons


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89470 entries, 0 to 89469
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gameId                   89470 non-null  object 
 1   team                     89470 non-null  object 
 2   year_x                   89470 non-null  int64  
 3   round_x                  89470 non-null  object 
 4   playerId                 89470 non-null  int64  
 5   displayName_x            89470 non-null  object 
 6   gameNumber               89470 non-null  int64  
 7   Disposals                89470 non-null  int64  
 8   Kicks                    89470 non-null  int64  
 9   Marks                    89470 non-null  int64  
 10  Handballs                89470 non-null  int64  
 11  Goals                    89470 non-null  int64  
 12  Behinds                  89470 non-null  int64  
 13  Hit Outs                 89470 non-null  int64  
 14  Tackles               

### Storing data for future use

In [7]:
year_list = sorted(list(df['year_x'].unique()))
year_list

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

## Cleaning Data:

In [8]:
df['rainfall'] = df['rainfall'].astype(float)
df['rainfall'].loc[df['rainfall']>0]

69      4.20
70      4.20
71      4.20
72      4.20
73      4.20
        ... 
89443   1.60
89444   1.60
89445   1.60
89446   1.60
89447   1.60
Name: rainfall, Length: 39430, dtype: float64

In [9]:
df['% Played'] = df['% Played'] / 100
df['% Played']

0       0.80
1       0.23
2       0.93
3       0.84
4       0.90
        ... 
89465   0.28
89466   0.79
89467   0.88
89468   0.83
89469   0.89
Name: % Played, Length: 89470, dtype: float64

### Converting date and time objects to datetime:

In [10]:
df = df[['playerId', 'height', 'weight', 'dob', 'position', 'gameId', 'team',
         'year_x', 'round_x', 'Disposals', 'Kicks', 'Marks', 'Handballs', 
         'Goals', 'Behinds', 'Hit Outs', 'Tackles', 'Rebounds', 'Inside 50s', 
         'Clearances', 'Clangers', 'Frees', 'Frees Against', 'Brownlow Votes', 
         'Contested Possessions', 'Uncontested Possessions', 'Contested Marks', 
         'Marks Inside 50', 'One Percenters', 'Bounces', 'Goal Assists', '% Played',
         'Subs', 'gameNumber',  'date', 'venue', 'startTime', 'attendance',
         'homeTeam', 'homeTeamScore', 'awayTeam', 'awayTeamScore', 'rainfall']]

df.rename({'year_x': 'year', 'round_x': 'round'}, axis=1, inplace=True)

In [11]:
date_dict = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
             'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [12]:
dob = df['dob'].str.split('-', expand=True)
dob[1] = df['dob'].str.split('-', expand=True)[1].replace(date_dict)
df['dob'] = dob.apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
df['dob']

0        12-02-2002
1        22-09-1992
2        31-12-1999
3         1-03-1997
4        20-11-1996
            ...    
89465    27-06-1989
89466    11-05-1993
89467    11-02-1993
89468    24-10-1992
89469     4-09-1989
Name: dob, Length: 89470, dtype: object

In [13]:
date = df['date'].str.split('-', expand=True)
date[1] = df['date'].str.split('-', expand=True)[1].replace(date_dict)
df['date'] = date.apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
df['date']

0        20-03-2021
1        20-03-2021
2        20-03-2021
3        20-03-2021
4        20-03-2021
            ...    
89465    02-09-2012
89466    02-09-2012
89467    02-09-2012
89468    02-09-2012
89469    02-09-2012
Name: date, Length: 89470, dtype: object

In [14]:
df['dob'] = pd.to_datetime(df['dob'], format='%d'+'-'+'%m'+'-'+'%Y')
df['date'] = pd.to_datetime(df['date'], format='%d'+'-'+'%m'+'-'+'%Y')

### Converting startTime to Start Time Hour and Start Time Minute

In [15]:
df['Start Time Hour'] = pd.to_datetime(df['startTime'], format= '%I:%M %p').dt.hour
df['Start Time Minute'] = pd.to_datetime(df['startTime'], format= '%I:%M %p').dt.minute

df.drop('startTime', axis=1, inplace=True)
df[['Start Time Hour', 'Start Time Minute']]

,Start Time Hour,Start Time Minute
0,16,5
1,16,5
2,16,5
3,16,5
4,16,5
...,...,...
89465,16,40
89466,16,40
89467,16,40
89468,16,40


### Fixing 'round' column by making Finals rounds numerical

In [16]:
round_list = {'QF': '24', 'SF': '25', 'PF': '26', 'GF': '27', 'EF': '24'}
df['round'] = df['round'].replace(round_list)
df['round'] = df['round'].str.replace('R', '').astype(int)

### Adding 'year+1' column for future indexing

In [17]:
df['year+1'] = df['year'] + 1
df['year+1']

0        2022
1        2022
2        2022
3        2022
4        2022
         ... 
89465    2013
89466    2013
89467    2013
89468    2013
89469    2013
Name: year+1, Length: 89470, dtype: int64

### Constructing index

In [18]:
df = df.set_index(['year','round'], append=True)

## Feature Creation:
### Age

In [19]:
df.insert(2, 'Age', (df['date'] - df['dob']) / np.timedelta64(1,'Y'))

df.drop('dob', axis=1, inplace=True)

### Total Points Scored 

In [20]:
df['Total Points Scored'] = df['Goals'] * 6 + df['Behinds']
df['Total Points Scored']

       year  round
0      2021  1         1
1      2021  1         0
2      2021  1         0
3      2021  1         0
4      2021  1        14
                      ..
89465  2012  23        0
89466  2012  23        0
89467  2012  23        0
89468  2012  23        6
89469  2012  23        0
Name: Total Points Scored, Length: 89470, dtype: int64

### Is Home Team

In [21]:
df['Is Home Team'] = np.where(df['team'] == df['homeTeam'], 1, 0)
df['Is Home Team']

       year  round
0      2021  1        1
1      2021  1        1
2      2021  1        1
3      2021  1        1
4      2021  1        1
                     ..
89465  2012  23       0
89466  2012  23       0
89467  2012  23       0
89468  2012  23       0
89469  2012  23       0
Name: Is Home Team, Length: 89470, dtype: int32

### Opponent Team

In [22]:
df['Opponent Team'] = np.where(df['team'] == df['homeTeam'], df['awayTeam'], df['homeTeam'])

df.drop('awayTeam', axis=1, inplace=True)

### Player Team Score and Opponent Team Score

In [23]:
df['Player Team Score'] = np.where(df['team'] == df['homeTeam'], df['homeTeamScore'], df['awayTeamScore'])
df['Opponent Team Score'] = np.where(df['team'] == df['homeTeam'], df['awayTeamScore'], df['homeTeamScore'])

### Score Margin (from the perspective of each player)

In [24]:
df['Margin'] = np.where(df['Is Home Team'] == 1,
                        df['homeTeamScore'] - df['awayTeamScore'],
                        df['awayTeamScore'] - df['homeTeamScore'])
df['Margin']

       year  round
0      2021  1        12
1      2021  1        12
2      2021  1        12
3      2021  1        12
4      2021  1        12
                      ..
89465  2012  23      -67
89466  2012  23      -67
89467  2012  23      -67
89468  2012  23      -67
89469  2012  23      -67
Name: Margin, Length: 89470, dtype: int64

### Game Result

In [25]:
conditions = [df['Margin'] > 0, df['Margin'] == 0, df['Margin'] < 0]
choices = [1, .5, 0]
df['Game Result'] = np.select(conditions, choices)
df['Game Result']

       year  round
0      2021  1       1.00
1      2021  1       1.00
2      2021  1       1.00
3      2021  1       1.00
4      2021  1       1.00
                     ... 
89465  2012  23      0.00
89466  2012  23      0.00
89467  2012  23      0.00
89468  2012  23      0.00
89469  2012  23      0.00
Name: Game Result, Length: 89470, dtype: float64

### Month

In [26]:
df['Month'] = df['date'].dt.month

### Day of year

In [27]:
df['Day of Year'] = df['date'].dt.dayofyear

### Weekday

In [28]:
df['Weekday'] = df['date'].dt.weekday

In [29]:
df.drop('date', axis=1, inplace=True)

### Spliting 'position' into multiple columns

In [30]:
df['position'].unique()

array(['Midfield, Forward', 'Defender', 'Forward', 'Midfield', 'Ruck',
       'Defender, Midfield', 'Forward, Ruck', 'Defender, Forward',
       'Defender, Ruck', 'Midfield, Ruck'], dtype=object)

In [31]:
position_list = ['Midfield', 'Forward', 'Defender', 'Ruck']

for pos in position_list:
    df[f'{pos}'] = np.where(df['position'].str.contains(pos), 1, 0)
    
df.drop('position', axis=1, inplace=True)

df[position_list].sum()

Midfield    31786
Forward     33602
Defender    34426
Ruck         5850
dtype: int64

### Shifting (offsetting) and averaging certain features by AFL round

In [32]:
df_final = df.copy()

In [33]:
shift_columns = ['Disposals', 'Kicks', 'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit Outs',
                 'Tackles', 'Rebounds', 'Inside 50s', 'Clearances', 'Clangers', 'Frees',
                 'Frees Against', 'Brownlow Votes', 'Contested Possessions', 'Uncontested Possessions',
                 'Contested Marks', 'Marks Inside 50', 'One Percenters', 'Bounces', 'Goal Assists',
                 '% Played', 'Subs', 'venue', 'Player Team Score', 'Opponent Team Score', 'rainfall', 
                 'Margin', 'Total Points Scored', 'Game Result', 'gameNumber', 'Start Time Hour', 
                 'Start Time Minute', 'Month', 'Weekday', 'Day of Year', 'team', 'Opponent Team']

average_and_sum_columns = [col for col in shift_columns if col not in ['venue', 'rainfall', 'Subs', 'Opponent Team',
                                                                       'team', 'Start Time Hour', 'Start Time Minute',
                                                                       'Month', 'Weekday', 'Day of Year']]

In [34]:
def offset_function(dataframe, column_list, year_list):
    frame = None
    for col in column_list:
        for i in range(1,6): # Shifting back 1 to 5 games
            for n, year in enumerate(year_list):
                year_df = dataframe.xs(year, level=1, drop_level=False)
                year_df = year_df.sort_index(level=['year', 'round'])
                df_group = year_df.groupby('playerId')[col]
                df_shift = df_group.shift(i)
                if n != 0:
                    df = pd.concat([df, df_shift])
                else:
                    df = df_shift
                    
            df = df.rename(f'{col}_shift_{i}')
            if frame is None:
                frame = df
            else:
                frame = pd.concat((frame, df), axis=1)
    return frame

In [35]:
offset_df = offset_function(df, shift_columns, year_list)
df_final = pd.concat((df_final, offset_df), axis=1)
offset_df

,,,Disposals_shift_1,Disposals_shift_2,Disposals_shift_3,Disposals_shift_4,Disposals_shift_5,Kicks_shift_1,Kicks_shift_2,Kicks_shift_3,Kicks_shift_4,Kicks_shift_5,Marks_shift_1,Marks_shift_2,Marks_shift_3,Marks_shift_4,Marks_shift_5,Handballs_shift_1,Handballs_shift_2,Handballs_shift_3,Handballs_shift_4,Handballs_shift_5,Goals_shift_1,Goals_shift_2,Goals_shift_3,Goals_shift_4,Goals_shift_5,Behinds_shift_1,Behinds_shift_2,Behinds_shift_3,Behinds_shift_4,Behinds_shift_5,Hit Outs_shift_1,Hit Outs_shift_2,Hit Outs_shift_3,Hit Outs_shift_4,Hit Outs_shift_5,Tackles_shift_1,Tackles_shift_2,Tackles_shift_3,Tackles_shift_4,Tackles_shift_5,Rebounds_shift_1,Rebounds_shift_2,Rebounds_shift_3,Rebounds_shift_4,Rebounds_shift_5,Inside 50s_shift_1,Inside 50s_shift_2,Inside 50s_shift_3,Inside 50s_shift_4,Inside 50s_shift_5,Clearances_shift_1,Clearances_shift_2,Clearances_shift_3,Clearances_shift_4,Clearances_shift_5,Clangers_shift_1,Clangers_shift_2,Clangers_shift_3,Clangers_shift_4,Clangers_shift_5,Frees_shift_1,Frees_shift_2,Frees_shift_3,Frees_shift_4,Frees_shift_5,Frees Against_shift_1,Frees Against_shift_2,Frees Against_shift_3,Frees Against_shift_4,Frees Against_shift_5,Brownlow Votes_shift_1,Brownlow Votes_shift_2,Brownlow Votes_shift_3,Brownlow Votes_shift_4,Brownlow Votes_shift_5,Contested Possessions_shift_1,Contested Possessions_shift_2,Contested Possessions_shift_3,Contested Possessions_shift_4,Contested Possessions_shift_5,Uncontested Possessions_shift_1,Uncontested Possessions_shift_2,Uncontested Possessions_shift_3,Uncontested Possessions_shift_4,Uncontested Possessions_shift_5,Contested Marks_shift_1,Contested Marks_shift_2,Contested Marks_shift_3,Contested Marks_shift_4,Contested Marks_shift_5,Marks Inside 50_shift_1,Marks Inside 50_shift_2,Marks Inside 50_shift_3,Marks Inside 50_shift_4,Marks Inside 50_shift_5,One Percenters_shift_1,One Percenters_shift_2,One Percenters_shift_3,One Percenters_shift_4,One Percenters_shift_5,Bounces_shift_1,Bounces_shift_2,Bounces_shift_3,Bounces_shift_4,Bounces_shift_5,Goal Assists_shift_1,Goal Assists_shift_2,Goal Assists_shift_3,Goal Assists_shift_4,Goal Assists_shift_5,% Played_shift_1,% Played_shift_2,% Played_shift_3,% Played_shift_4,% Played_shift_5,Subs_shift_1,Subs_shift_2,Subs_shift_3,Subs_shift_4,Subs_shift_5,venue_shift_1,venue_shift_2,venue_shift_3,venue_shift_4,venue_shift_5,Player Team Score_shift_1,Player Team Score_shift_2,Player Team Score_shift_3,Player Team Score_shift_4,Player Team Score_shift_5,Opponent Team Score_shift_1,Opponent Team Score_shift_2,Opponent Team Score_shift_3,Opponent Team Score_shift_4,Opponent Team Score_shift_5,rainfall_shift_1,rainfall_shift_2,rainfall_shift_3,rainfall_shift_4,rainfall_shift_5,Margin_shift_1,Margin_shift_2,Margin_shift_3,Margin_shift_4,Margin_shift_5,Total Points Scored_shift_1,Total Points Scored_shift_2,Total Points Scored_shift_3,Total Points Scored_shift_4,Total Points Scored_shift_5,Game Result_shift_1,Game Result_shift_2,Game Result_shift_3,Game Result_shift_4,Game Result_shift_5,gameNumber_shift_1,gameNumber_shift_2,gameNumber_shift_3,gameNumber_shift_4,gameNumber_shift_5,Start Time Hour_shift_1,Start Time Hour_shift_2,Start Time Hour_shift_3,Start Time Hour_shift_4,Start Time Hour_shift_5,Start Time Minute_shift_1,Start Time Minute_shift_2,Start Time Minute_shift_3,Start Time Minute_shift_4,Start Time Minute_shift_5,Month_shift_1,Month_shift_2,Month_shift_3,Month_shift_4,Month_shift_5,Weekday_shift_1,Weekday_shift_2,Weekday_shift_3,Weekday_shift_4,Weekday_shift_5,Day of Year_shift_1,Day of Year_shift_2,Day of Year_shift_3,Day of Year_shift_4,Day of Year_shift_5,team_shift_1,team_shift_2,team_shift_3,team_shift_4,team_shift_5,Opponent Team_shift_1,Opponent Team_shift_2,Opponent Team_shift_3,Opponent Team_shift_4,Opponent Team_shift_5
,year,round,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [36]:
def window_function(dataframe, column_list, year_list):
    frame = None
    for col in column_list:
        for window_func in ('boxcar', 'exponential', 'triang'):
            for func in ('sum', 'mean'):
                for i in (3, 5): # Rolling window size
                    for n, year in enumerate(year_list):
                        year_df = dataframe.xs(year, level=1, drop_level=False)
                        year_df = year_df.sort_index(level=['year', 'round'])
                        df_group = year_df.groupby('playerId')[col]
                        df_rolling = df_group.rolling(i, win_type=window_func, min_periods=1)
                        if func == 'sum':
                            df_rolling = df_rolling.sum()
                        elif func == 'mean':
                            df_rolling = df_rolling.mean()

                        df_shift = df_rolling.shift()
                        if n != 0:
                            df = pd.concat([df, df_shift])
                        else:
                            df = df_shift

                    df = df.rename(f'{col}_{window_func}_{func}_{i}')
                    if frame is None:
                        frame = df
                    else:
                        frame = pd.concat((frame, df), axis=1)
    frame = frame.reset_index('playerId', drop=True)
    return frame

In [37]:
window_df = window_function(df, average_and_sum_columns, year_list)
df_final = pd.concat((df_final, window_df), axis=1)
window_df

,,,Disposals_boxcar_sum_3,Disposals_boxcar_sum_5,Disposals_boxcar_mean_3,Disposals_boxcar_mean_5,Disposals_exponential_sum_3,Disposals_exponential_sum_5,Disposals_exponential_mean_3,Disposals_exponential_mean_5,Disposals_triang_sum_3,Disposals_triang_sum_5,Disposals_triang_mean_3,Disposals_triang_mean_5,Kicks_boxcar_sum_3,Kicks_boxcar_sum_5,Kicks_boxcar_mean_3,Kicks_boxcar_mean_5,Kicks_exponential_sum_3,Kicks_exponential_sum_5,Kicks_exponential_mean_3,Kicks_exponential_mean_5,Kicks_triang_sum_3,Kicks_triang_sum_5,Kicks_triang_mean_3,Kicks_triang_mean_5,Marks_boxcar_sum_3,Marks_boxcar_sum_5,Marks_boxcar_mean_3,Marks_boxcar_mean_5,Marks_exponential_sum_3,Marks_exponential_sum_5,Marks_exponential_mean_3,Marks_exponential_mean_5,Marks_triang_sum_3,Marks_triang_sum_5,Marks_triang_mean_3,Marks_triang_mean_5,Handballs_boxcar_sum_3,Handballs_boxcar_sum_5,Handballs_boxcar_mean_3,Handballs_boxcar_mean_5,Handballs_exponential_sum_3,Handballs_exponential_sum_5,Handballs_exponential_mean_3,Handballs_exponential_mean_5,Handballs_triang_sum_3,Handballs_triang_sum_5,Handballs_triang_mean_3,Handballs_triang_mean_5,Goals_boxcar_sum_3,Goals_boxcar_sum_5,Goals_boxcar_mean_3,Goals_boxcar_mean_5,Goals_exponential_sum_3,Goals_exponential_sum_5,Goals_exponential_mean_3,Goals_exponential_mean_5,Goals_triang_sum_3,Goals_triang_sum_5,Goals_triang_mean_3,Goals_triang_mean_5,Behinds_boxcar_sum_3,Behinds_boxcar_sum_5,Behinds_boxcar_mean_3,Behinds_boxcar_mean_5,Behinds_exponential_sum_3,Behinds_exponential_sum_5,Behinds_exponential_mean_3,Behinds_exponential_mean_5,Behinds_triang_sum_3,Behinds_triang_sum_5,Behinds_triang_mean_3,Behinds_triang_mean_5,Hit Outs_boxcar_sum_3,Hit Outs_boxcar_sum_5,Hit Outs_boxcar_mean_3,Hit Outs_boxcar_mean_5,Hit Outs_exponential_sum_3,Hit Outs_exponential_sum_5,Hit Outs_exponential_mean_3,Hit Outs_exponential_mean_5,Hit Outs_triang_sum_3,Hit Outs_triang_sum_5,Hit Outs_triang_mean_3,Hit Outs_triang_mean_5,Tackles_boxcar_sum_3,Tackles_boxcar_sum_5,Tackles_boxcar_mean_3,Tackles_boxcar_mean_5,Tackles_exponential_sum_3,Tackles_exponential_sum_5,Tackles_exponential_mean_3,Tackles_exponential_mean_5,Tackles_triang_sum_3,Tackles_triang_sum_5,Tackles_triang_mean_3,Tackles_triang_mean_5,Rebounds_boxcar_sum_3,Rebounds_boxcar_sum_5,Rebounds_boxcar_mean_3,Rebounds_boxcar_mean_5,Rebounds_exponential_sum_3,Rebounds_exponential_sum_5,Rebounds_exponential_mean_3,Rebounds_exponential_mean_5,Rebounds_triang_sum_3,Rebounds_triang_sum_5,Rebounds_triang_mean_3,Rebounds_triang_mean_5,Inside 50s_boxcar_sum_3,Inside 50s_boxcar_sum_5,Inside 50s_boxcar_mean_3,Inside 50s_boxcar_mean_5,Inside 50s_exponential_sum_3,Inside 50s_exponential_sum_5,Inside 50s_exponential_mean_3,Inside 50s_exponential_mean_5,Inside 50s_triang_sum_3,Inside 50s_triang_sum_5,Inside 50s_triang_mean_3,Inside 50s_triang_mean_5,Clearances_boxcar_sum_3,Clearances_boxcar_sum_5,Clearances_boxcar_mean_3,Clearances_boxcar_mean_5,Clearances_exponential_sum_3,Clearances_exponential_sum_5,Clearances_exponential_mean_3,Clearances_exponential_mean_5,Clearances_triang_sum_3,Clearances_triang_sum_5,Clearances_triang_mean_3,Clearances_triang_mean_5,Clangers_boxcar_sum_3,Clangers_boxcar_sum_5,Clangers_boxcar_mean_3,Clangers_boxcar_mean_5,Clangers_exponential_sum_3,Clangers_exponential_sum_5,Clangers_exponential_mean_3,Clangers_exponential_mean_5,Clangers_triang_sum_3,Clangers_triang_sum_5,Clangers_triang_mean_3,Clangers_triang_mean_5,Frees_boxcar_sum_3,Frees_boxcar_sum_5,Frees_boxcar_mean_3,Frees_boxcar_mean_5,Frees_exponential_sum_3,Frees_exponential_sum_5,Frees_exponential_mean_3,Frees_exponential_mean_5,Frees_triang_sum_3,Frees_triang_sum_5,Frees_triang_mean_3,Frees_triang_mean_5,Frees Against_boxcar_sum_3,Frees Against_boxcar_sum_5,Frees Against_boxcar_mean_3,Frees Against_boxcar_mean_5,Frees Against_exponential_sum_3,Frees Against_exponential_sum_5,Frees Against_exponential_mean_3,Frees Against_exponential_mean_5,Frees Against_triang_sum_3,Frees Against_triang_sum_5,Frees Against_triang_mean_3,F

In [38]:
def current_year_cumulative_function(dataframe, column_list, year_list):
    frame = None
    for col in column_list:
        for func in ['sum', 'max', 'min']:
            for n, year in enumerate(year_list):
                year_df = dataframe.xs(year, level=1, drop_level=False)
                year_df = year_df.set_index('playerId', append=True)
                year_df = year_df.sort_index(level=['year', 'round'])
                df_group = year_df.groupby('playerId')[col]
                if func == 'sum':
                    df_cum = df_group.cumsum()
                elif func == 'max':
                    df_cum = df_group.cummax()
                elif func == 'min':
                    df_cum = df_group.cummin()
                
                df_group_cum = df_cum.groupby('playerId')
                df_shift = df_group_cum.shift()
                    
                if n != 0:
                    df = pd.concat([df, df_shift])
                else:
                    df = df_shift
            df = df.rename(f'{col}_cumulative_{func}')
            if frame is None:
                frame = df
            else:
                frame = pd.concat((frame, df), axis=1)
    frame = frame.reset_index('playerId', drop=True)
    return frame

In [39]:
cumulative_df = current_year_cumulative_function(df, average_and_sum_columns, year_list)
df_final = pd.concat((df_final, cumulative_df), axis=1)
cumulative_df

,,,Disposals_cumulative_sum,Disposals_cumulative_max,Disposals_cumulative_min,Kicks_cumulative_sum,Kicks_cumulative_max,Kicks_cumulative_min,Marks_cumulative_sum,Marks_cumulative_max,Marks_cumulative_min,Handballs_cumulative_sum,Handballs_cumulative_max,Handballs_cumulative_min,Goals_cumulative_sum,Goals_cumulative_max,Goals_cumulative_min,Behinds_cumulative_sum,Behinds_cumulative_max,Behinds_cumulative_min,Hit Outs_cumulative_sum,Hit Outs_cumulative_max,Hit Outs_cumulative_min,Tackles_cumulative_sum,Tackles_cumulative_max,Tackles_cumulative_min,Rebounds_cumulative_sum,Rebounds_cumulative_max,Rebounds_cumulative_min,Inside 50s_cumulative_sum,Inside 50s_cumulative_max,Inside 50s_cumulative_min,Clearances_cumulative_sum,Clearances_cumulative_max,Clearances_cumulative_min,Clangers_cumulative_sum,Clangers_cumulative_max,Clangers_cumulative_min,Frees_cumulative_sum,Frees_cumulative_max,Frees_cumulative_min,Frees Against_cumulative_sum,Frees Against_cumulative_max,Frees Against_cumulative_min,Brownlow Votes_cumulative_sum,Brownlow Votes_cumulative_max,Brownlow Votes_cumulative_min,Contested Possessions_cumulative_sum,Contested Possessions_cumulative_max,Contested Possessions_cumulative_min,Uncontested Possessions_cumulative_sum,Uncontested Possessions_cumulative_max,Uncontested Possessions_cumulative_min,Contested Marks_cumulative_sum,Contested Marks_cumulative_max,Contested Marks_cumulative_min,Marks Inside 50_cumulative_sum,Marks Inside 50_cumulative_max,Marks Inside 50_cumulative_min,One Percenters_cumulative_sum,One Percenters_cumulative_max,One Percenters_cumulative_min,Bounces_cumulative_sum,Bounces_cumulative_max,Bounces_cumulative_min,Goal Assists_cumulative_sum,Goal Assists_cumulative_max,Goal Assists_cumulative_min,% Played_cumulative_sum,% Played_cumulative_max,% Played_cumulative_min,Player Team Score_cumulative_sum,Player Team Score_cumulative_max,Player Team Score_cumulative_min,Opponent Team Score_cumulative_sum,Opponent Team Score_cumulative_max,Opponent Team Score_cumulative_min,Margin_cumulative_sum,Margin_cumulative_max,Margin_cumulative_min,Total Points Scored_cumulative_sum,Total Points Scored_cumulative_max,Total Points Scored_cumulative_min,Game Result_cumulative_sum,Game Result_cumulative_max,Game Result_cumulative_min,gameNumber_cumulative_sum,gameNumber_cumulative_max,gameNumber_cumulative_min
,year,round,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
80362,2012,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80363,2012,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80364,2012,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80365,2012,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80366,2012,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [40]:
def previous_year_cumulative_function(dataframe, column_list, year_list):
    df = dataframe.reset_index(['year', 'round'])
    df = df.set_index(['playerId', 'year+1'])
    df_sum = df.groupby(['playerId', 'year+1'])[average_and_sum_columns].sum()
    df_mean = df.groupby(['playerId', 'year+1'])[average_and_sum_columns].mean()
    return df_sum, df_mean

In [41]:
df_sum, df_mean = previous_year_cumulative_function(df, average_and_sum_columns, year_list)

In [42]:
df_final = df_final.reset_index(['year', 'round'])

In [43]:
df_final = df_final.merge(df_sum, how='left', left_on=['playerId', 'year'],
                          right_on=['playerId', 'year+1'], suffixes=('', '_prevous_year_sum'))
df_sum

Disposals  Kicks  Marks  Handballs  Goals  Behinds  \
playerId    year+1                                                       
1993707514  2013          289    244     88         45      1        2   
            2014          198    151     63         47      0        0   
            2015          279    198    103         81      1        0   
            2016          116     80     35         36      1        1   
1996722607  2013          520    285     99        235     35        9   
...                       ...    ...    ...        ...    ...      ...   
20151001643 2018          265    139     60        126     14        8   
            2019          338    232     92        106     48       22   
            2020          285    204     81         81     34       22   
            2021          135     86     26         49     14       12   
            2022          420    251    108        169     23       23   

                    Hit Outs  Tackles  Rebounds  Inside 50s  Clearances  \
playerId    year+1                                                        
1993707514  2013           0       20        89          20           5   
            2014           0       14        67           8           1   
            2015           0       11        60          21           3   
            2016           0        6        36           5           0   
1996722607  2013           0       47        30          83          52   
...                      ...      ...       ...         ...         ...   
20151001643 2018           1       47        10          46          34   
            2019           0       48         9          72          39   
            2020           0       35         3          61          33   
            2021           0       12         5          28          17   
            2022           0       39        24          70          61   

                    Clangers  Frees  Frees Against  Brownlow Votes  \
playerId    year+1                                                   
1993707514  2013          34     12             10               1   
            2014          21      0              9               0   
            2015          23      9             12               0   
            2016           8      4              3               0   
1996722607  2013          54     20             19               4   
...                      ...    ...            ...             ...   
20151001643 2018          25     10              9               5   
            2019          53     16             15              12   
            2020          48     18             14               4   
            2021          19      6              4               5   
            2022          61     12             18               9   

                    Contested Possessions  Uncontested Possessions  \
playerId    year+1                                                   
1993707514  2013                       86                      151   
            2014                       36                      132   
            2015                       58                      205   
            2016                       34                       78   
1996722607  2013                      147                      369   
...                                   ...                      ...   
20151001643 2018                      102                      170   
            2019                      133                      207   
            2020                      126                      152   
            2021                       65                       71   
            2022                      140                      288   

                    Contested Marks  Marks Inside 50  One Percenters  Bounces  \
playerId    year+1                                                              
1993707514  2013                 12                0             110        0   
            2014

In [44]:
df_final = df_final.merge(df_mean, how='left', left_on=['playerId', 'year'],
                          right_on=['playerId', 'year+1'], suffixes=('', '_prevous_year_mean'))
df_mean

Disposals  Kicks  Marks  Handballs  Goals  Behinds  \
playerId    year+1                                                       
1993707514  2013        16.06  13.56   4.89       2.50   0.06     0.11   
            2014        15.23  11.62   4.85       3.62   0.00     0.00   
            2015        18.60  13.20   6.87       5.40   0.07     0.00   
            2016        16.57  11.43   5.00       5.14   0.14     0.14   
1996722607  2013        22.61  12.39   4.30      10.22   1.52     0.39   
...                       ...    ...    ...        ...    ...      ...   
20151001643 2018        18.93   9.93   4.29       9.00   1.00     0.57   
            2019        16.10  11.05   4.38       5.05   2.29     1.05   
            2020        16.76  12.00   4.76       4.76   2.00     1.29   
            2021        13.50   8.60   2.60       4.90   1.40     1.20   
            2022        21.00  12.55   5.40       8.45   1.15     1.15   

                    Hit Outs  Tackles  Rebounds  Inside 50s  Clearances  \
playerId    year+1                                                        
1993707514  2013        0.00     1.11      4.94        1.11        0.28   
            2014        0.00     1.08      5.15        0.62        0.08   
            2015        0.00     0.73      4.00        1.40        0.20   
            2016        0.00     0.86      5.14        0.71        0.00   
1996722607  2013        0.00     2.04      1.30        3.61        2.26   
...                      ...      ...       ...         ...         ...   
20151001643 2018        0.07     3.36      0.71        3.29        2.43   
            2019        0.00     2.29      0.43        3.43        1.86   
            2020        0.00     2.06      0.18        3.59        1.94   
            2021        0.00     1.20      0.50        2.80        1.70   
            2022        0.00     1.95      1.20        3.50        3.05   

                    Clangers  Frees  Frees Against  Brownlow Votes  \
playerId    year+1                                                   
1993707514  2013        1.89   0.67           0.56            0.06   
            2014        1.62   0.00           0.69            0.00   
            2015        1.53   0.60           0.80            0.00   
            2016        1.14   0.57           0.43            0.00   
1996722607  2013        2.35   0.87           0.83            0.17   
...                      ...    ...            ...             ...   
20151001643 2018        1.79   0.71           0.64            0.36   
            2019        2.52   0.76           0.71            0.57   
            2020        2.82   1.06           0.82            0.24   
            2021        1.90   0.60           0.40            0.50   
            2022        3.05   0.60           0.90            0.45   

                    Contested Possessions  Uncontested Possessions  \
playerId    year+1                                                   
1993707514  2013                     4.78                     8.39   
            2014                     2.77                    10.15   
            2015                     3.87                    13.67   
            2016                     4.86                    11.14   
1996722607  2013                     6.39                    16.04   
...                                   ...                      ...   
20151001643 2018                     7.29                    12.14   
            2019                     6.33                     9.86   
            2020                     7.41                     8.94   
            2021                     6.50                     7.10   
            2022                     7.00                    14.40   

                    Contested Marks  Marks Inside 50  One Percenters  Bounces  \
playerId    year+1                                                              
1993707514  2013               0.67             0.00            6.11     0.00   
            2014

### Dropping redundant or irrelevant columns

In [45]:
df_final.drop(['homeTeamScore', 'awayTeamScore', 'playerId', 'gameId'], axis=1, inplace=True)

### Dropping columns of data that would be unavailable the day of the game

In [46]:
df_final.drop(['Disposals', 'Kicks', 'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit Outs', 
               'Tackles', 'Rebounds', 'Inside 50s', 'Clearances', 'Clangers', 'Frees', 
               'Frees Against', 'Brownlow Votes', 'Contested Possessions', 'Bounces',
               'Uncontested Possessions', 'Contested Marks', 'Marks Inside 50', 'One Percenters',
               '% Played', 'Subs', 'Total Points Scored',  'Goal Assists', 'Player Team Score',
               'Opponent Team Score', 'Margin', 'Game Result', 'attendance', 'rainfall'],
               axis=1, inplace=True)

### Normalizing columns by round and by game

In [47]:
list(df_final.columns)

['year',
 'round',
 'height',
 'Age',
 'weight',
 'team',
 'gameNumber',
 'venue',
 'homeTeam',
 'Start Time Hour',
 'Start Time Minute',
 'year+1',
 'Is Home Team',
 'Opponent Team',
 'Month',
 'Day of Year',
 'Weekday',
 'Midfield',
 'Forward',
 'Defender',
 'Ruck',
 'Disposals_shift_1',
 'Disposals_shift_2',
 'Disposals_shift_3',
 'Disposals_shift_4',
 'Disposals_shift_5',
 'Kicks_shift_1',
 'Kicks_shift_2',
 'Kicks_shift_3',
 'Kicks_shift_4',
 'Kicks_shift_5',
 'Marks_shift_1',
 'Marks_shift_2',
 'Marks_shift_3',
 'Marks_shift_4',
 'Marks_shift_5',
 'Handballs_shift_1',
 'Handballs_shift_2',
 'Handballs_shift_3',
 'Handballs_shift_4',
 'Handballs_shift_5',
 'Goals_shift_1',
 'Goals_shift_2',
 'Goals_shift_3',
 'Goals_shift_4',
 'Goals_shift_5',
 'Behinds_shift_1',
 'Behinds_shift_2',
 'Behinds_shift_3',
 'Behinds_shift_4',
 'Behinds_shift_5',
 'Hit Outs_shift_1',
 'Hit Outs_shift_2',
 'Hit Outs_shift_3',
 'Hit Outs_shift_4',
 'Hit Outs_shift_5',
 'Tackles_shift_1',
 'Tackles_shift_

In [48]:
normalization_list = df_final.select_dtypes(exclude=object).columns

In [49]:
def normalization_func(dataframe, column_list):
    try:
        for col in column_list:
            df_round = dataframe.groupby(['year', 'round'])[col]
            dataframe[f'{col}_round_norm'] = df_round.transform(lambda x: (x - min(x))/(max(x)-min(x)))

            df_game = dataframe.groupby(['year', 'round', 'homeTeam'])[col]
            dataframe[f'{col}_game_norm'] = df_game.transform(lambda x: (x - min(x))/(max(x)-min(x)))
        return dataframe
    except:
        print(col)

In [50]:
df_final = normalization_func(df_final, normalization_list)
df_final

year  round  height   Age  weight              team  gameNumber  \
0      2021      1     181 19.10      81          Adelaide           1   
1      2021      1     181 28.49      81          Adelaide         168   
2      2021      1     198 21.22      83          Adelaide           3   
3      2021      1     188 24.05      88          Adelaide          31   
4      2021      1     200 24.33      90          Adelaide           9   
...     ...    ...     ...   ...     ...               ...         ...   
89465  2012     23     189 23.18      79  Western Bulldogs           8   
89466  2012     23     181 19.31      85  Western Bulldogs          16   
89467  2012     23     194 19.56      94  Western Bulldogs           4   
89468  2012     23     186 19.86      85  Western Bulldogs          25   
89469  2012     23     187 23.00      86  Western Bulldogs          42   

               venue        homeTeam  Start Time Hour  Start Time Minute  \
0      Adelaide Oval        Adelaide               16                  5   
1      Adelaide Oval        Adelaide               16                  5   
2      Adelaide Oval        Adelaide               16                  5   
3      Adelaide Oval        Adelaide               16                  5   
4      Adelaide Oval        Adelaide               16                  5   
...              ...             ...              ...                ...   
89465          Gabba  Brisbane Lions               16                 40   
89466          Gabba  Brisbane Lions               16                 40   
89467          Gabba  Brisbane Lions               16                 40   
89468          Gabba  Brisbane Lions               16                 40   
89469          Gabba  Brisbane Lions               16                 40   

       year+1  Is Home Team   Opponent Team  Month  Day of Year  Weekday  \
0        2022             1         Geelong      3           79        5   
1        2022             1         Geelong      3           79        5   
2        2022             1         Geelong      3           79        5   
3        2022             1         Geelong      3           79        5   
4        2022             1         Geelong      3           79        5   
...       ...           ...             ...    ...          ...      ...   
89465    2013             0  Brisbane Lions      9          246        6   
89466    2013             0  Brisbane Lions      9          246        6   
89467    2013             0  Brisbane Lions      9          246        6   
89468    2013             0  Brisbane Lions      9          246        6   
89469    2013             0  Brisbane Lions      9          246        6   

       Midfield  Forward  Defender  Ruck  Disposals_shift_1  \
0             1        1         0     0                NaN   
1             0        0         1     0                NaN   
2             0        0         1     0                NaN   
3             0        0         1     0                NaN   
4             0        1         0     0                NaN   
...         ...      ...       ...   ...                ...   
89465         0        1         0     0              17.00   
89466         0        1         0     0              14.00   
89467         0        0         1     0              11.00   
89468         0        1         0     0              29.00   
89469         0        0         1     0              16.00   

       Disposals_shift_2  Disposals_shift_3  Disposals_shift_4  \
0                    NaN                NaN                NaN   
1                    NaN                NaN                NaN   
2                    NaN                NaN                NaN   
3                    NaN                NaN                NaN   
4                    NaN                NaN                NaN   
...                  ...                ...                ...   
89465               8.00              12.00               8.00   
89466  

## Creating AFL Fantasy table based on relative point distributions
### AFL Fantasy:

In [52]:
afl_fantasy_points = {'Kicks': 3,
                    'Marks': 3,
                    'Handballs': 2,
                    'Goals': 6,
                    'Behinds': 1,
                    'Hit Outs': 1,
                    'Tackles': 4,
                    'Frees': 1,
                    'Frees Against': -3}

In [53]:
from functools import reduce

fantasy_list = []
for col_name, pts in afl_fantasy_points.items():
    col_vals = df[col_name] * pts
    df[col_name + '_fantasy'] = col_vals
    fantasy_list.append(col_vals)
df['Total Fantasy'] = reduce((lambda x, y: x + y), fantasy_list)